In [1]:
import PIL.Image as Image
import numpy as np
import tensorflow as tf
import os
import time

import sys
sys.path.append("../../model")
import l21RobustDeepAutoencoderOnST as l21RDA

sys.path.append("../../data")
import ImShow as I

def l21RDAE(X, layers, lamda, folder, learning_rate = 0.15, inner = 100, outer = 10, 
            batch_size = 133,re_init=False,inputsize = (28,28)):
    if not os.path.isdir(folder):
        os.makedirs(folder)
    os.chdir(folder)
    with tf.Graph().as_default():
        with tf.Session() as sess:
            rael21 = l21RDA.RobustL21Autoencoder(sess = sess, lambda_= lamda*X.shape[0], 
                                                 layers_sizes=layers)
            l21L, l21S = rael21.fit(X = X, sess = sess, inner_iteration = inner, iteration = outer, 
                                    batch_size = batch_size, learning_rate = learning_rate,  
                                    re_init=re_init,verbose = False)
            l21R = rael21.getRecon(X = X, sess = sess)
            l21H = rael21.transform(X, sess)
            Image.fromarray(I.tile_raster_images(X=l21S,img_shape=inputsize, tile_shape=(10, 10),tile_spacing=(1, 1))).save(r"l21S.png")
            Image.fromarray(I.tile_raster_images(X=l21R,img_shape=inputsize, tile_shape=(10, 10),tile_spacing=(1, 1))).save(r"l21R.png")
            Image.fromarray(I.tile_raster_images(X=l21L,img_shape=inputsize, tile_shape=(10, 10),tile_spacing=(1, 1))).save(r"l21L.png")
            l21S.dump("l21S.npk")
    os.chdir("../")

def compare_frame():

    X = np.load(r"../../data/data.npk",allow_pickle=True)

    inner = 50
    outer = 20



    lambda_list = [0.0001, 0.0003, 0.0008, 0.001, 0.0015, 0.00035, 0.00045, 
         0.00055, 0.00065, 0.00075, 0.00085, 0.00095, 0.00105, 0.00115, 0.00125]
#     lambda_list = [0.00015,0.00018,0.0002,0.00025,0.00028,0.0003]
    print(lambda_list)
    
    layers = [784, 400, 200] ## S trans
    print("start")
    start_time = time.time()
    image_X = Image.fromarray(I.tile_raster_images(X = X, img_shape = (28,28), tile_shape=(10, 10),tile_spacing=(1, 1)))
    image_X.save(r"X.png")
    for lam in lambda_list:
        folder = "lam" + str(lam)
        l21RDAE(X = X, layers=layers, lamda = lam, folder = folder, learning_rate = 0.001, 
                inner = inner, outer = outer, batch_size = 133,re_init=True,inputsize = (28,28))
        print("done: lam", str(lam))
    print ("Runing time:" + str(time.time() - start_time) + " s")
if __name__ == "__main__":
    compare_frame()


D:\SoftWare\Python\envs\py_env_36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\SoftWare\Python\envs\py_env_36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\SoftWare\Python\envs\py_env_36\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\SoftWare\Python\envs\py_env_36\lib\site-packages\tensorflow\python\framework

[0.0001, 0.0003, 0.0008, 0.001, 0.0015, 0.00035, 0.00045, 0.00055, 0.00065, 0.00075, 0.00085, 0.00095, 0.00105, 0.00115, 0.00125]
start
done: lam 0.0001
done: lam 0.0003
done: lam 0.0008
done: lam 0.001
done: lam 0.0015
done: lam 0.00035
done: lam 0.00045
done: lam 0.00055
done: lam 0.00065
done: lam 0.00075
done: lam 0.00085
done: lam 0.00095
done: lam 0.00105
done: lam 0.00115
done: lam 0.00125
Runing time:5463.8517537117 s
